<a href="https://colab.research.google.com/github/aimtyaem/EOInfo/blob/main/image_processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import tensorflow as tf  # Or PyTorch, depending on your model
import numpy as np
import matplotlib.pyplot as plt
import rasterio  # For reading raster images (GeoTIFFs)
from rasterio.transform import from_origin  # For creating GeoTIFFs
import cv2  # For image preprocessing

# --- 1. Load your trained model ---
model_path = "path/to/your/trained/model"  # Replace with the actual path
model = tf.keras.models.load_model(model_path)  # If using TensorFlow/Keras
# For PyTorch, you would load the model differently.


def preprocess_image(image_path):
    """Preprocesses the image for the model."""
    img = cv2.imread(image_path) # or however you're reading the image
    if img is None:
        raise ValueError(f"Could not read image at {image_path}")
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Important: Convert to RGB!
    img = cv2.resize(img, (224, 224))  # Resize to match your model's input size (example)
    img = img / 255.0  # Normalize pixel values (important!)
    img = np.expand_dims(img, axis=0)  # Add batch dimension (important!)
    return img


def predict_carbon_footprint(image_path):
    """Predicts carbon footprint from an image."""
    try:
      img = preprocess_image(image_path)
      predictions = model.predict(img)

      # --- 2. Process Predictions ---
      # Interpret the model's output.  This is highly specific to your model.
      # Example (for a classification model):
      predicted_class = np.argmax(predictions)
      # Get probabilities (if needed)
      probabilities = predictions

      # Example (for a regression model):
      predicted_carbon_value = predictions[0][0] # Assuming a single output value

      return predicted_carbon_value, probabilities # or predicted_class, probabilities

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return None, None

def visualize_and_save_prediction(image_path, prediction_value, probabilities=None):
    """Visualizes the prediction on the image and saves it as a Matplotlib figure."""

    img = cv2.imread(image_path) # Or however you are reading the image
    if img is None:
        print(f"Error: Could not read image at {image_path}")
        return

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # Convert to RGB for matplotlib
    plt.imshow(img)

    title = f"Predicted Carbon Footprint: {prediction_value:.2f}" # Format as needed
    if probabilities is not None:
        # Add probabilities to the title or as text on the plot if it's a classification problem
        title += f" (Probabilities: {probabilities})" # Example
    plt.title(title)
    plt.axis('off')  # Hide axes

    # --- 3. Save as Matplotlib figure ---
    output_path = "prediction_output.png"  # Or .jpg, .pdf, etc.
    plt.savefig(output_path)
    print(f"Prediction visualization saved to {output_path}")
    plt.show() # to show the image



# --- Example usage ---
image_path = "path/to/your/image.tif"  # Or .jpg, .png, etc.
predicted_carbon, probabilities = predict_carbon_footprint(image_path)

if predicted_carbon is not None:
    print(f"Predicted Carbon Footprint: {predicted_carbon}")
    visualize_and_save_prediction(image_path, predicted_carbon, probabilities)


# --- Example for processing multiple images in a directory ---
import os
image_dir = "path/to/your/image/directory"
for filename in os.listdir(image_dir):
    if filename.endswith(('.tif', '.jpg', '.png')): # Add more extensions if needed
        image_path = os.path.join(image_dir, filename)
        predicted_carbon, probabilities = predict_carbon_footprint(image_path)
        if predicted_carbon is not None:
            print(f"Image: {filename}, Predicted Carbon Footprint: {predicted_carbon}")
            visualize_and_save_prediction(image_path, predicted_carbon, probabilities)